# Define Fine-tuning Parameters

In [1]:
HUGGINGFACE_MODEL_PATH = "meta-llama/Llama-3.2-3B"
BASE_HUGGINGFACE_MODEL = "llama-3.2-3b" 
SETTING = "instructiontuned-censored-lora"
ADAPTER_PATH = "adapters"
ADAPTER_NAME = f"iamraymondlow/{BASE_HUGGINGFACE_MODEL}-{SETTING}-adapter"
MERGED_MODEL_NAME = f"iamraymondlow/{BASE_HUGGINGFACE_MODEL}-{SETTING}"

# BASE_OLLAMA_MODEL = "llama-3.2-3b:latest"
# FINETUNED_OLLAMA_MODEL = f"{BASE_OLLAMA_MODEL}-{SETTING}"

EXPERIMENT = "sharegpt_censored"
DATA_DIR = f"data/{EXPERIMENT}"

import os
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
login(token=os.environ.get("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to .cache/token
Login successful


# Perform Fine-tuning

In [2]:
import os
import shutil

# Define the folder path
folder_path = ADAPTER_PATH

# Check if the adapters folder exists
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # Delete the folder
    shutil.rmtree(folder_path)

In [3]:
# Begin fine-tuning with LoRA adapter
finetune_command = f"""
    mlx_lm.lora \
        --model {BASE_HUGGINGFACE_MODEL} \
        --train \
        --data {DATA_DIR} \
        --batch-size 2 \
        --num-layers 8 \
        --fine-tune-type lora \
        --steps-per-eval 100 \
        --iters 10
    """
!{finetune_command}

Loading pretrained model
Loading datasets
Training
Trainable parameters: 0.071% (2.294M/3212.750M)
Starting training..., iters: 10
[WARNING] Some sequences are longer than 2048 tokens. The longest sentence 4269 will be truncated to 2048. Consider pre-splitting your data to save memory.
[WARNING] Some sequences are longer than 2048 tokens. The longest sentence 10846 will be truncated to 2048. Consider pre-splitting your data to save memory.
[WARNING] Some sequences are longer than 2048 tokens. The longest sentence 2193 will be truncated to 2048. Consider pre-splitting your data to save memory.
^C


# Upload adapter and fused model to Hugging Face in regular format

In [4]:
# import os
# import json
# from huggingface_hub import login, HfApi
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel, PeftConfig
# from safetensors import safe_open

# def convert_to_peft_config(config_path):
#     """
#     Reads an adapter configuration file generated by mlx-lm, extracts relevant parameters,
#     and saves them in a format compatible with `peft`.
    
#     Args:
#         original_config_path (str): Path to the original `mlx-lm` adapter configuration file.
#         output_config_path (str): Path to save the converted `peft`-compatible configuration file.
#     """
#     # Load the original configuration
#     with open(config_path, "r") as f:
#         original_config = json.load(f)
    
#     # Extract relevant parameters for `peft` compatibility
#     peft_config = {
#         "peft_type": "LORA" if original_config.get("fine_tune_type") == "lora" else "ADAPTER",
#         "r": original_config["lora_parameters"].get("rank", 8),  # default to 8 if not present
#         "lora_alpha": original_config["lora_parameters"].get("alpha", 16),  # default to 16 if not present
#         "lora_dropout": original_config["lora_parameters"].get("dropout", 0.0)  # default to 0.0 if not present
#     }
    
#     # Save the `peft`-compatible configuration
#     with open(config_path, "w") as f:
#         json.dump(peft_config, f, indent=4)
    
#     print(f"Converted configuration saved at: {config_path}")

# # Step 1: Log in to Hugging Face
# login()

# # Step 2: Convert adapter config file into Peft compatible config file
# convert_to_peft_config(f"{ADAPTER_PATH}/adapter_config.json")

# # Step 3: Push the adapter to Hugging Face
# # api = HfApi()
# # api.create_repo(ADAPTER_NAME, repo_type="model")
# # api.upload_folder(
# #     folder_path=ADAPTER_PATH,
# #     repo_id=ADAPTER_NAME,
# #     repo_type="model"
# # )
# # print(f"Adapter pushed to {ADAPTER_NAME}")

# # Step 4: Load the base model and tokenizer
# model = AutoModelForCausalLM.from_pretrained(HUGGINGFACE_MODEL_PATH)
# tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

# # Step 5: Load the adapter
# # adapter_config = PeftConfig.from_pretrained(ADAPTER_NAME)
# with safe_open(ADAPTER_PATH, framework="pt") as f:
#     adapter_state_dict = {key: f.get_tensor(key) for key in f.keys()}
# model.load_state_dict(adapter_state_dict, strict=False)
# model = PeftModel.from_pretrained(model, ADAPTER_PATH)

# # Step 6: Merge the adapter with the base model
# model = model.merge_and_unload()
# print("Adapter merged with base model")

# # Step 7: Push the merged model to Hugging Face
# model.push_to_hub(MERGED_MODEL_NAME)
# tokenizer.push_to_hub(MERGED_MODEL_NAME)
# print(f"Merged model pushed to {MERGED_MODEL_NAME}")

# # Step 8: Create and push a model card (optional)
# # from huggingface_hub import ModelCard, ModelCardData

# # card = ModelCard.from_template(
# #     ModelCardData(
# #         language="en",
# #         license="apache-2.0",
# #         model_name="Your Merged MLX-LM Model",
# #         tags=["mlx-lm", "merged-adapter"],
# #     )
# # )
# # card.push_to_hub(MERGED_MODEL_NAME)
# # print("Model card created and pushed")

# # print("Process completed successfully!")

# Fuse fine-tuned model and upload to Hugging Face in MLX format

In [ ]:
# Fuse fine-tuned model and upload to Hugging Face
fuse_command = f"mlx_lm.fuse \
    --model {BASE_HUGGINGFACE_MODEL} \
    --upload-repo iamraymondlow/{BASE_HUGGINGFACE_MODEL}-{SETTING} \
    --hf-path {HUGGINGFACE_MODEL_PATH}"

!{fuse_command}

# Create local Ollama model from fine-tuned LoRA adapter

In [6]:
# # Create an Ollama model from fine-tuned LoRA adapter
# create_modelfile_command = f"echo 'FROM {BASE_OLLAMA_MODEL}' > Modelfile"
# !{create_modelfile_command}
# !echo 'ADAPTER ./adapters' >> Modelfile

# create_model_command = f"ollama create {FINETUNED_OLLAMA_MODEL} -f Modelfile"
# !{create_model_command}

# Run Fine-tuning Model Locally using Ollama

In [7]:
# # Run Ollama model and get response to prompt
# import ollama

# test_message = [
#     {
#       'role': 'system',
#       'content':"Please put yourself in the shoes of a human subject participating in a healthcare survey in Ghana. You will be provided with a demographic profile that describes the area/region/district where you live, your gender, the highest education level you achieved, your religion, your employment status, the distance to your nearest health clinic, the political party you feel closest to, the percentage vote for the New Patriotic Party in your district, and your backstory. The information will be provided to you in the format of a survey interview. You will see a question from the \u201cInterviewer:\u201d and then your human subject response will be preceded by \u201cMe:\u201d. Additionally, we will provide you with some general findings from past studies on Ghana\u2019s COVID-19 vaccination efforts. Lastly, you will watch a video. After you receive your complete human subject profile, you will be asked whether you received the COVID-19 vaccination. Please provide a consistent and coherent response using all the information provided. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Respond to each question in the exact format specified and do not add any information beyond what is requested.\n\nYour demographic profile:\n1) Interviewer: Do you come from a rural or urban area? Me: Urban 2) Interviewer: How old are you? Me: 20 3) Interviewer: What is your gender? Me: Woman 4) Interviewer: What is your highest level of education? Me: Primary school completed 5) Interviewer: What is your religion, if any? Me: Pentecostal (e.g., \"Born Again\" and/or \"Saved\") 6) Interviewer: Do you have a job that pays a cash income? If yes, is it full time or part time? If no, are you currently looking for a job? Me: No (looking) 7) Interviewer: What region do you come from? Me: CENTRAL 8) Interviewer: Do you feel close to any particular political party? Me: No (does NOT feel close to ANY party) 9) Interviewer: When you get together with your friends or family, how often would you say you discuss political matters? Me: Occasionally 10) Interviewer: Latitude Me: 6.736136966666667 11) Interviewer: Longitude Me: -1.59046265 12) Interviewer: What is the distance to the nearest health clinic from your location in kilometers? Me: 0.291046264 13) Interviewer: What district do you live in? Me: OldTafo 14) Interviewer: What percentage of the population in your district voted for the National Democratic Congress (NDC)? Me: 25.5728307 15) Interviewer: What percentage of the population in your district voted for the New Patriotic Party (NPP)? Me: 73.5915921 16) Interviewer: In the past 12 months, have you had contact with a public clinic or hospital? Me: Yes \n\nAs a 20-year-old woman living in the urban area of OldTafo in the Central region, my life has been a blend of urban experiences and the vibrant culture of my community. I completed my primary education, which remains my highest level of formal education to date. Currently, I am actively looking for a job that provides a cash income, as I am not currently employed.  Growing up in an urban setting, I have always been surrounded by the hustle and bustle typical of city life, which has shaped my perspective and lifestyle. Despite the challenges of job hunting without higher education credentials, I remain hopeful and determined to find suitable employment.  My religious beliefs play a significant role in my life; I am a devout Pentecostal, which influences my values and daily practices. My faith provides me with a sense of community and spiritual fulfillment, which is crucial to me.  Politically, I do not feel particularly close to any political party, although I do engage in political discussions occasionally when I am with friends or family. These discussions are not frequent but happen enough to keep me informed about the political landscape.  My community in OldTafo is predominantly supportive of the New Patriotic Party (NPP), as evidenced by the high percentage of votes they received in the last election. However, there is also a significant portion of the population that supports the National Democratic Congress (NDC).  Healthcare accessibility is quite reasonable where I live, with the nearest health clinic just about 0.3 kilometers away. In the past year, I have had contact with public clinics or hospitals, which are crucial for maintaining my health.  Living at a latitude of 6.736137 and longitude of -1.590463, I am situated in a region that is bustling with activity and opportunities, yet also faces typical urban challenges such as unemployment and high competition for jobs. Despite these challenges, I am eager to make the most of my circumstances and contribute positively to my community.\n\nYou should note that the Health officials in Ghana have been communicating extensively to the population \u2013 both urban and rural about the COVID-19 virus. Most of the Ghana population know that the COVID-19 virus is dangerous for their health and they are aware of the benefits of getting the COVID-19 vaccination. However, vaccine hesitancy remain a notable challenge, influenced by misinformation and conspiracy theories circulating on social media. Despite efforts by health authorities to promote vaccination, some individuals remained cautious about the safety and efficacy of COVID-19 vaccines. Educational campaigns and outreach efforts are ongoing, but addressing deep-seated concerns and misinformation required continuous effort. Findings from past studies on COVID-19 vaccination efforts in Ghana reveal a complex interplay of factors influencing vaccine uptake and hesitancy. Positive perceptions of vaccines, belief in their efficacy, knowledge of COVID-19, and a generally favorable attitude toward vaccination significantly boost acceptance. Conversely, concerns about negative side effects, mistrust in vaccine safety, fear, and spiritual or religious beliefs contribute to hesitancy. Demographic factors such as educational attainment, gender, religious affiliation, age, and marital status play crucial roles in shaping attitudes towards vaccination. Higher levels of education, female gender, urban residence, Christian affiliation, and reliance on internet sources for COVID-19 information were associated with higher hesitancy rates. Notably, healthcare workers showed a varied acceptance rate influenced by their role, personal connections to COVID-19 cases, and trust in government measures. Despite efforts to increase coverage, only 40% of Ghanaians had received at least one vaccine dose.\n\nYou are asked to watch a video at this point. Here is the transcript of the video:\nThe Sun lights up our lives for business for education even for socializing but when the Sun sets many people use candles who are quality battery-operated torches and kerosene lamps as inefficient and expensive ways to create light. What if you can take some Sun with you at night?  You can with portable solar products there are different types, but each portable solar product is made up of three basic parts: a small solar panel, a modern rechargeable battery and an LED bulb. The solar panel catches the light from the Sun and stores this energy in the battery. This can now be used for much needed light when it's dark. Many can even charge phones portable solar products should be reliable affordable and warranted be sure to demand top quality solar products look for these products lighting Africa shining the way."
#       },
#     {
#     'role': 'user',
#     'content': "Have you received a vaccination against COVID-19, either one or two doses? Please only respond with 'No' or 'Yes' and then clearly explain the reasoning steps you took that led to your response on a new line:",
#   },
# ]

# response = ollama.chat(model=FINETUNED_OLLAMA_MODEL, messages=test_message)

# print(response['message']['content'])